In [13]:
import os
import cv2
import glob
from tqdm import tqdm
from face_detector import FaceDetector

# --- 설정 ---
NUM_IMAGES_TO_PROCESS = 10000
INPUT_FOLDER = '/content/img_align_celeba'
OUTPUT_FOLDER = '/content/cropped_celeba_10k'

# --- 얼굴 탐지기 생성 및 결과 폴더 생성 ---
face_detector = FaceDetector()
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- 메인 처리 루프 ---
# [수정된 부분] 하위 폴더까지 모두 검색하도록 수정
# '/**/'는 모든 하위 폴더를 의미하며, recursive=True 옵션이 필요합니다.
print("이미지 파일 목록을 검색합니다 (하위 폴더 포함)...")
image_files = sorted(glob.glob(f'{INPUT_FOLDER}/**/*.jpg', recursive=True))
files_to_process = image_files[:NUM_IMAGES_TO_PROCESS]

print(f"\n총 {len(image_files)}개의 이미지 중 {len(files_to_process)}개를 처리합니다...")

total_faces_cropped = 0
for img_path in tqdm(files_to_process):
    original_image = cv2.imread(img_path)
    if original_image is None:
        continue

    cropped_faces, _ = face_detector.run(original_image, target_size=640, score_threshold=0.9)

    base_name = os.path.basename(img_path)
    for i, face_img in enumerate(cropped_faces):
        output_filename = f"face_{i}_{base_name}"
        output_path = os.path.join(OUTPUT_FOLDER, output_filename)
        cv2.imwrite(output_path, face_img)
        total_faces_cropped += 1

print(f"\n작업 완료! 총 {total_faces_cropped}개의 얼굴 이미지를 '{OUTPUT_FOLDER}' 폴더에 저장했습니다.")

이미지 파일 목록을 검색합니다 (하위 폴더 포함)...

총 202599개의 이미지 중 10000개를 처리합니다...


100%|██████████| 10000/10000 [07:42<00:00, 21.60it/s]


작업 완료! 총 9830개의 얼굴 이미지를 '/content/cropped_celeba_10k' 폴더에 저장했습니다.
